### Baseline для хакатона Rutube по задаче "Теггирование видео"

В рамках данного ноутбука мы рассмотрим наивный подход к решению поставленной задачи: векторный поиск навания видео в базе векторов тегов.

В конце есть пример получения sample_submission.csv - пример файла, который нужно загрузить на лидерборд.


In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from tqdm.autonotebook import tqdm
import numpy as np
import faiss

#### Берем данные с id видео и его названием, также загружаем иерархические теги

In [24]:
data = pd.read_csv("../test/x_test.csv", index_col=0)[['video_id', 'title', 'description']]
taxonomy = pd.read_csv("IAB_tags.csv")

print(data.columns)
print(data.head(5))

print(taxonomy.head(5))
print(taxonomy.columns)

Index(['video_id', 'title', 'description'], dtype='object')
                             video_id  \
353  85c6ac4a4cfc3778cd2e65139d970dba   
985  7eecbd10bdaa596c2921ecb9bf7dbf69   
877  0d7b83b4d141158214acf42ab2ca0519   
983  cf3ef0b2d6227ad372a9b7dcb6cb2df3   
31   c093daa4f0e96d8f2e4736c9240898f3   

                                                 title  \
353                         Салют, Начальник | 4 серия   
985  Сколько Стоит Тачка: Галина Ржаксенская - от п...   
877                       Какова Меркурия? | Выпуск 12   
983                             Смехмашина | Выпуск 13   
31     Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышева   

                                           description  
353  На очередной сельской планёрке обсуждают насущ...  
985  В этом выпуске вы узнаете, на чем ездит блогер...  
877  Женя вместе с астрологом попробуют разобраться...  
983  Подвели итоги розыгрыша. Поздравляем счастливо...  
31   Героиней сегодняшнего выпуска Абьюз-шоу стала ...  
  

#### Для создания эмбеддинга берем русскоязычный Берт и загружаем в sentence transformer, который позволяет создавать эмбеддинг для всего предложения и сам обрезает его до максимально возможного числа токенов

In [25]:
model = SentenceTransformer('nizamovtimur/multilingual-e5-large-videotags')
dim = 1024 # размер вектора эмбеддинга

c:\git\rutube-video-tagging\.venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--nizamovtimur--multilingual-e5-large-videotags. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [26]:
model.save("../saved_models/multilingual-e5-large-videotags")

#### Создаем эмбеддинги для названий видео

In [27]:
data['UD'] = data['title'] + data['description']
data['embedding'] = data['UD'].apply(lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

#### Создаем векторы для тегов:
 Для каждого 1 уровня иерархии в отдельности и для следующих уровней формата уровень 1: уровень 2: уровень 3 

In [28]:
def get_tags():
    tags = {}
    for i, row in tqdm(taxonomy.iterrows()):
        if isinstance(row['Уровень 1 (iab)'], str):
            tags[row['Уровень 1 (iab)']] = model.encode(row['Уровень 1 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 2 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 3 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
    return tags

tags = get_tags()
tags_list = list(tags.keys())
vectors = np.array(list(tags.values()))

0it [00:00, ?it/s]

611it [05:39,  1.80it/s]


### Создаем векторную базу faiss для эффективного векторного поиска

In [29]:
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
print(index.ntotal)
index.add(vectors)
print(index.ntotal)

0
610


#### Смотрим несколько получившихся примеров 
Генерим по 3 близких предсказания для каждого названия видео

In [30]:
topn = 3
scores, predictions = index.search(np.array(data['embedding'].to_list()[:10]), topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags_list)[predictions[j]])
    print("SAMPLE", data['title'].to_list()[:10][j])
    print("\n")

SCORES [0.6357642  0.6302114  0.61655825]
PREDICTION_by_title ['Фильмы и анимация: Комедии' 'Телевидение: Мультфильмы и анимация'
 'Фильмы и анимация']
SAMPLE Салют, Начальник | 4 серия


SCORES [0.6278831 0.4237215 0.3736378]
PREDICTION_by_title ['Личные финансы' 'Массовая культура: Семьи знаменитостей'
 'Массовая культура: Дома знаменитостей']
SAMPLE Сколько Стоит Тачка: Галина Ржаксенская - от подаренных прав до холостяка


SCORES [0.7887893  0.47634828 0.31725204]
PREDICTION_by_title ['Религия и духовность: Астрология' 'Наука: Космос и астрономия'
 'Хобби и интересы: Коллекционирование: Марки и монеты']
SAMPLE Какова Меркурия? | Выпуск 12


SCORES [0.6927234 0.5768223 0.5662971]
PREDICTION_by_title ['Массовая культура: Юмор и сатира' 'Фильмы и анимация: Комедии'
 'Телевидение: Комедийные каналы']
SAMPLE Смехмашина | Выпуск 13


SCORES [0.7317821  0.61411965 0.59934074]
PREDICTION_by_title ['Массовая культура: Беременности знаменитостей'
 'Медицина: Медицинские направления: Сексолог

#### Для формирования sample_submission будем брать только наилучшее предсказания для каждого видео
Сейчас у вас уже есть sample_submission с нужными для скоринга video_id, но пока нет информации о видео, она появится ближе к концу хакатона
Для примера прогоним через весь train dataset

In [31]:
topn=1
sample_submission = pd.DataFrame(data=data['video_id'].to_list(), columns=['video_id'])
sample_submission['predicted_tags']=np.nan
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype('object')

for i, row in data.iterrows():
    scores, predictions = index.search(np.array([row['embedding']]), topn)
    index_i = sample_submission[sample_submission.video_id == row.video_id].index
    sample_submission.at[index_i[0], 'predicted_tags'] = [tags_list[predictions[0][0]]] # вытаскиваем предсказание из

In [32]:
print(sample_submission.head(5))


                           video_id  \
0  85c6ac4a4cfc3778cd2e65139d970dba   
1  7eecbd10bdaa596c2921ecb9bf7dbf69   
2  0d7b83b4d141158214acf42ab2ca0519   
3  cf3ef0b2d6227ad372a9b7dcb6cb2df3   
4  c093daa4f0e96d8f2e4736c9240898f3   

                                    predicted_tags  
0                     [Фильмы и анимация: Комедии]  
1                                 [Личные финансы]  
2               [Религия и духовность: Астрология]  
3               [Массовая культура: Юмор и сатира]  
4  [Массовая культура: Беременности знаменитостей]  


#### В predicted_tags нужно записывать list тегов, например ['Карьера: Cтажировки', 'Карьера: Составление резюме'] или ['Массовая культура: Сериалы']

In [33]:
sample_submission.to_csv("sample_submission.csv", index_label=0)